Prep

In [1]:
import flywheel
import re
import sys
import os
import glob
import shutil
import nibabel as nb
from fw_heudiconv.cli import export

/storage/ttapera/fw-heudiconv/fw_heudiconv/backend_funcs/query.py:4: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  from nibabel.nicom.dicomwrappers import wrapper_from_data


200526-17:44:05,495 nipype.utils INFO:
	 Running nipype version 1.4.0 (latest: 1.4.2)


INFO: Running nipype version 1.4.0 (latest: 1.4.2)


Initialise flywheel data:

In [ ]:
client = flywheel.Client()

RBC_proj = client.projects.find_first('label="ReproBrainChart"')

RBC_subjects = RBC_proj.subjects()

PNC_CS_proj = client.projects.find_first('label="PNC_CS_810336"')
PNC_LG_proj = client.projects.find_first('label="PNC_LG_810336"')

PNC_CS_subjects = PNC_CS_proj.subjects()
PNC_LG_subjects = PNC_LG_proj.subjects()

# get the penn_ids list
penn_ids = pd.read_csv('/storage/ttapera/RBC/data/bblid_renamed.csv', dtype = {'bblid': str, 'reid': str})

penn_ids['sub_id'] = 'sub-' + penn_ids['reid']

In [ ]:
# function to remove a subject from the RBC project
def remove_current_rbc_subject(row, RBC_subjects=RBC_subjects):

    target = [x for x in RBC_subjects if x.label == row['sub_id']]
    if len(target) == 1:
        print('found subject to remove: ', list(row), target[0].id)
        print('client.delete_subject(target[0].id)')
        return True
    else:
        return False


In [ ]:
penn_ids['process'] = penn_ids.apply(remove_current_rbc_subject, axis = 1)

In [ ]:
penn_ids.to_csv('/storage/ttapera/RBC/data/bblids_2.csv', index=False)

In [ ]:
sessions = None
#downloads = export.gather_bids(client, PNC_CS_proj.label, '80010', sessions)

In [ ]:
sessions = None

for index, row in tqdm(penn_ids.iterrows(), total=penn_ids.shape[0]):
    
    print("Processing...")
    should_replace = False
    # first remove the existing data on RBC
    should_replace = remove_current_rbc_subject(row, RBC_subjects)
    
    if should_replace:
        # download the data from PNC
        print("Gathering PNC CS BIDS...")
        pnc1_data = export.gather_bids(client, PNC_CS_proj.label, row['bblid'], sessions)        
        print("\nDownloading BIDS...\n")
        export.download_bids(
            client, pnc1_data, 
            '/storage/ttapera/RBC/data/', 
            folders_to_download = ['anat', 'func', 'fmap'],
            dry_run=False,
            name='PNC1')

        print("Gathering PNC LG BIDS...")
        try:
            pnc2_data = export.gather_bids(client, PNC_LG_proj.label, row['bblid'], sessions)

            export.download_bids(
                client, pnc2_data, 
                '/storage/ttapera/RBC/data/', 
                folders_to_download = ['anat', 'func', 'fmap'],
                dry_run=False,
                name='PNC2')
        except:
            print("no second session!")
        
        # merge any and all data
        print("n\Merging\n")
        os.system("cp -r -R /storage/ttapera/RBC/data/PNC*/* /storage/ttapera/RBC/data/upload/")
        
        # rename the files:
        print("\nRenaming\n")
        for root, subdirs, files in os.walk("/storage/ttapera/RBC/data/upload/"):

            for f in files:
                if row['bblid'] in f:
                    print("renaming file ", f)
                    new_name = f.replace(row['bblid'], row['reid'])
                    os.rename(root + "/" + f, root + "/" + new_name)

        # rename the subject-level directory
        old_name = "/storage/ttapera/RBC/data/upload/sub-" + row['bblid']
        new_name = old_name.replace(row['bblid'], row['reid'])
        shutil.move(old_name, new_name)        
        os.system("rm -rf /storage/ttapera/RBC/data/upload/sub*/sub*")
        
        # apply defacing here
        print("\nDefacing\n")
        for root, subdirs, files in os.walk("/storage/ttapera/RBC/data/upload/"):

            for f in files:
                if ".nii" in f:
                    print("defacing " + f)
                    cmd = ("docker run -ti --rm "
                           "-v {}:/data "
                           "pennbbl/mrideface "
                           "/data/{} "
                           "/opt/mrideface/talairach_mixed_with_skull.gca "
                           "/opt/mrideface/face.gca "
                           "/data/{} ".format(root, f, f))
                    
                    print(cmd)
                    output = subprocess.check_output(cmd)
        
        # upload here
        print("\nUploading\n")
        
        # remove from filesystem here
        print("\nClearing local data\n")
        os.system("rm -rf /storage/ttapera/RBC/data/upload/*")
        os.system("rm -rf /storage/ttapera/RBC/data/PNC*")
        os.system("rm -rf /storage/ttapera/RBC/data/sub*")

In [16]:
check_download(path)

Checking for successful download in  /storage/ttapera/RBC/data/test/sub-*/ses-*/*/*.nii.gz
/storage/ttapera/RBC/data/test/sub-80010/ses-PNC2/func/sub-80010_ses-PNC2_task-frac2back.nii.gz
/storage/ttapera/RBC/data/test/sub-80010/ses-PNC2/func/sub-80010_ses-PNC2_task-idemo.nii.gz
/storage/ttapera/RBC/data/test/sub-80010/ses-PNC2/func/sub-80010_ses-PNC2_task-rest_acq-singleband_bold.nii.gz
/storage/ttapera/RBC/data/test/sub-80010/ses-PNC2/fmap/sub-80010_ses-PNC2_phase1.nii.gz
/storage/ttapera/RBC/data/test/sub-80010/ses-PNC2/fmap/sub-80010_ses-PNC2_magnitude1.nii.gz
/storage/ttapera/RBC/data/test/sub-80010/ses-PNC2/fmap/sub-80010_ses-PNC2_magnitude2.nii.gz
/storage/ttapera/RBC/data/test/sub-80010/ses-PNC2/fmap/sub-80010_ses-PNC2_phase2.nii.gz


True

In [73]:
def download_bids2(args):

    print("Gathering PNC CS BIDS...")

    processes = []
    
    processes.append("fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_CS_810336 --subject {} --data-type anat".format(str(args[1])))
    
    processes.append("fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_CS_810336 --subject {} --data-type func".format(str(args[1])))
    
    processes.append("fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_CS_810336 --subject {} --data-type fmap".format(str(args[1])))
    
    
    processes.append("fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_LG_810336 --subject {} --data-type anat".format(str(args[1])))
    
    processes.append("fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_LG_810336 --subject {} --data-type func".format(str(args[1])))
    
    processes.append("fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_LG_810336 --subject {} --data-type fmap".format(str(args[1])))

    statuses = []
    
    for process in processes:
        p = subprocess.Popen(process, shell=True)
        statuses.append(p)

    for i, p in enumerate(statuses):
        stat = p.wait()
        print("Process ", i, " complete with status ", stat)

In [43]:
args = ['foo', '80010', 'foofie']

In [15]:
def check_download(path):

    print("Checking for successful download in ", path)
    
    paths = glob.glob(path)
    paths = paths[1:]
    if len(paths) <= 1:
        return False
    for name in paths:

        print(name)
        try:
            img = nb.load(name)
        except Exception as e:
            print(e)
            return False

    return True

In [44]:
download_bids2(args)

Gathering PNC CS BIDS...
Process  0  complete with status  0
Process  1  complete with status  0
Process  2  complete with status  0
Process  3  complete with status  0
Process  4  complete with status  0
Process  5  complete with status  0


In [65]:
def cleanup_bids(path):
    
    paths = glob.glob(path)
    paths = paths[1:]

    for name in paths:

        print(name)
        pattern = re.compile("sub-[0-9]+_ses-PNC[^1]+_task-[^(rest)]")
        if pattern.search(name):
            print("removing file...")
            os.system("rm {}".format(name))


In [70]:
def rename_bids(args):
    
    for root, subdirs, files in os.walk("/storage/ttapera/RBC/data/bids_dataset/"):

        for f in files:
            if args[1] in f:
                print("renaming file ", f)
                new_name = f.replace(args[1], args[2])
                os.rename(root + "/" + f, root + "/" + new_name)
    
    old_name = "/storage/ttapera/RBC/data/bids_dataset/sub-" + args[1]
    new_name = old_name.replace(args[1], args[2])
    shutil.move(old_name, new_name)

In [71]:
rename_bids(args)

In [83]:
def download_bids2(args):
    
    # use the fw export bids cli + subprocess to download the anat fmap and func folders
    print("Gathering PNC BIDS...")

    processes = []
    
    processes.append("fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_CS_810336 --subject {} --data-type anat".format(str(args[1])))
    
    processes.append("fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_CS_810336 --subject {} --data-type func".format(str(args[1])))
    
    processes.append("fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_CS_810336 --subject {} --data-type fmap".format(str(args[1])))
    
    processes.append("fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_CS_810336 --subject {} --data-type dwi".format(str(args[1])))
    
    processes.append("fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_LG_810336 --subject {} --data-type anat".format(str(args[1])))
    
    processes.append("fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_LG_810336 --subject {} --data-type func".format(str(args[1])))
    
    processes.append("fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_LG_810336 --subject {} --data-type fmap".format(str(args[1])))

    # use subprocess just to see which download successfully; those without PNC_LG will fail with exit 1
    statuses = []
    
    for process in processes:
        p = subprocess.Popen(process, shell=True)
        statuses.append(p)

    for i, p in enumerate(statuses):
        stat = p.wait()
        if stat == 1:
            print("Failed export:\n", processes[i])
        else:
            print("Process ", i, " complete with status ", stat)
        
def cleanup_bids(path):
    
    # Remove any files from PNC2/3 that are not 
    paths = glob.glob(path)
    paths = paths[1:]

    for name in paths:

        print(name)
        pattern = re.compile("sub-[0-9]+_ses-PNC[^1]+_task-[^(rest)]")
        if pattern.search(name):
            print("removing file...")
            os.system("rm {}".format(name))

def rename_bids(args):
    
    for root, subdirs, files in os.walk("/storage/ttapera/RBC/data/bids_dataset/"):

        for f in files:
            if args[1] in f:
                print("renaming file ", f)
                new_name = f.replace(args[1], args[2])
                os.rename(root + "/" + f, root + "/" + new_name)
    
    old_name = "/storage/ttapera/RBC/data/bids_dataset/sub-" + args[1]
    new_name = old_name.replace(args[1], args[2])
    shutil.move(old_name, new_name)

def check_headers(path):

    print("Checking for successful download in ", path)
    
    paths = glob.glob(path)
    paths = paths[1:]
    if len(paths) <= 1:
        return False
    for name in paths:

        print(name)
        try:
            img = nb.load(name)
        except Exception as e:
            print(e)
            return False

    return True

args = ['foo', '139469', 'foofie', 'sub-foofie']
path = "/storage/ttapera/RBC/data/bids_dataset/sub-*/ses-*/*/*"

download_bids2(args)
cleanup_bids(path)
rename_bids(args)
check = check_headers(path)
print(check)

Gathering PNC BIDS...
Process  0  complete with status  0
Failed export:
 fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_CS_810336 --subject 139469 --data-type func
Process  2  complete with status  0
Process  3  complete with status  0
Failed export:
 fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_LG_810336 --subject 139469 --data-type anat
Failed export:
 fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_LG_810336 --subject 139469 --data-type func
Failed export:
 fw export bids /storage/ttapera/RBC/data/bids_dataset --project PNC_LG_810336 --subject 139469 --data-type fmap
/storage/ttapera/RBC/data/bids_dataset/sub-139469/ses-PNC1/anat/sub-139469_ses-PNC1_T1w.json
/storage/ttapera/RBC/data/bids_dataset/sub-139469/ses-PNC1/fmap/sub-139469_ses-PNC1_magnitude2.nii.gz
/storage/ttapera/RBC/data/bids_dataset/sub-139469/ses-PNC1/fmap/sub-139469_ses-PNC1_phase2.nii.gz
/storage/ttapera/RBC/data/bids_dataset/sub-139469/ses-PNC1/fmap/s